In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pymorphy2
import matplotlib as mpl



## Очистка данных

Так как при парсинге могут возникнуть ошибки, то очистим текстовые данные от пропусков, эмодзи, непонятных знаков и тд

In [ ]:
data = pd.read_csv('data.csv')

In [ ]:
data.head(3)

### Удалим пропуски и сверхкороткие строки

In [ ]:
i=len(data)-1
while i >0: 
    print(i)
    text=str(data['text'][i])
    sign=str(data['sign'][i])
    date=str(data['date'][i])
    if len(sign)<2:
        data=data.drop([i])
        print('deleted sign',i)
    if len(text)<2:
        data=data.drop([i])
        print('deleted text',i)
    if len(date)>13:
        data=data.drop([i])
        print('deleted date',i)
    if date[0]!="2":
        data=data.drop([i])
        print('deleted num2',i)
    i-=1 

chars = set([c for c in text]) # создадим множество из слов, т.е. словарь


### Удалим из текста все, что не текст 

In [ ]:
dataset=np.array(data)

In [ ]:
for i in range(dataset.shape[0]):
    text=str(dataset[i][2])
    text=text.replace('.','')
    text=text.replace(',','')
    text=text.replace(')','')
    text=text.replace('(','')
    text=text.replace('»','')
    text=text.replace('«','')
    text=text.replace('"','')
    text=text.replace(':','')
    text=text.replace('-',' ')
    text=text.replace('!','')
    text=text.lower()
    dataset[i][2]=text

In [ ]:
dataset

## Создадим словарь из слов датасета, приведенных к именительному падежу

In [ ]:
raw_words=set()
morph = pymorphy2.MorphAnalyzer()

In [ ]:
for i in range(dataset.shape[0]):
    text=str(dataset[i][2])
    text_chars=text.split(' ')
    for  word in text_chars:
        raw_words.add(word)

## Сохраним множество слов

In [ ]:
words_list=[]
for word in raw_words:
    words_list.append({'word':word})

words_df=pd.DataFrame(words_list)

In [ ]:
#words_df.to_csv('words.csv')

## Посчитаем количество повторений 

То есть посчитаем для каждого слова в каком предложении и сколько раз потворялось. Такой массив имеет размерность 

(количество_предложений, размер_словаря)

In [ ]:
words=pd.read_csv('words.csv')

In [ ]:
words.head()

In [ ]:
chars=np.array(words)

In [ ]:
chars_set=set()
for i,word in enumerate(chars):
    text=morph.parse(word[0])[0].normal_form
    chars_set.add(text)

In [ ]:
signs_set=set()
for line in dataset:
    signs_set.add(line[1])
signs_list=[sign for sign in signs_set]
chars_list=[char for char in chars_set]

In [ ]:
stat_data=np.zeros((len(chars_list),len(signs_list)))
for i,line in enumerate(dataset):
    sign=line[1]
    text=str(line[2])
    text=text.split(' ')
    for word in text:
        if word=='':
            continue
        fixed=morph.parse(word)[0].normal_form
        x=chars_list.index(fixed)
        y=signs_list.index(sign)
        stat_data[x][y]+=1

In [ ]:
np.save('stat_data', stat_data)
np.save('signs_list', signs_list)
np.save('chars_list', chars_list)

## Выведем полученные статистические данные на диаграмму

Статистика получилась интересная. 
Например, для всех знаков зодиака повторяются первые 10+-2 слов. Также после набора "повторяющихся" слова идет один определенный знак зодиака. Для каждого знака он свой. Это можно интерпретироват как то, что либо у всех знаков есть свой "подходящий знак", либо то,что есть уже заготовленные правила написания гороскопов.  

In [ ]:
stat_data = np.load('parsedArrays/stat_data.npy')
signs = np.load('parsedArrays/signs_list.npy')

stat_data_list=list(stat_data)
chars_list = list(np.load('parsedArrays/chars_list.npy'))
signs={'aries':0,'gemini':1,'taurus':2,'cancer':3,'leo':4,'virgo':5,'libra':6,
       'scorpio':7,'sagittarius':8,'capricorn':9,'aquarius':10,'pisces':11}

### Можно поиграться с другими знаками

Для этого нужно поменять переменную sign

In [ ]:
sign='aries'  # можно тут поменять знак зодиака на другой, чтобы вывести соответсвующую диаграмму
groups = [i for i in range(len(chars_list))]
counts = np.array([num[signs[sign]] for num in stat_data])
counts_list=list(counts)
sum=counts.sum(axis=0)

In [ ]:
counts.sort(axis=0)
N=100
common_vals=counts[::-1]
common_vals=common_vals[:N]

In [ ]:
common_indeces=[]

for val in common_vals:
    key=counts_list.index(val)
    common_indeces.append(key)
    
common_words=[]

for index in common_indeces:
    common_words.append(chars_list[index])

In [ ]:
dpi = 150
fig = plt.figure(dpi = dpi, figsize = (1024 / dpi, 2048 / dpi) )
mpl.rcParams.update({'font.size': 9})

plt.title('Относительная частота появления слов')

ax = plt.axes()
ax.xaxis.grid(True, zorder = 1)

x=range(N)
y=common_vals/sum

plt.barh(x, y, height = 0.2, color = 'red', label=sign ,alpha = 0.7, zorder =2)
plt.yticks(range(len(common_words)), common_words, rotation = 1)
plt.legend(loc='upper right')
